In [11]:
import sys
sys.path.append("../src/")

import pandas as pd
import re
import pickle as pkl
import datetime as dt
from razdel import tokenize, sentenize
from string import punctuation
from vocabulary import Vocabulary


def get_date(url):
    dates = re.findall(r"\d\d\d\d\/\d\d\/\d\d", url)
    return next(iter(dates), None)


def get_texts(dataset):
    texts = []
    for text in dataset["text"]:
        for sentence in sentenize(text):
            texts.append([token.text.lower() for token in tokenize(sentence.text) if token.text not in punctuation])

    for title in dataset["title"]:
        texts.append([token.text.lower() for token in tokenize(title) if token.text not in punctuation])
    return texts


def build_contexts(tokenized_texts, vocabulary, window_size):
    contexts = []
    for tokens in tokenized_texts:
        for i in range(len(tokens)):
            central_word = vocabulary.get_index(tokens[i])
            context = [
                vocabulary.get_index(tokens[i + delta])
                for delta in range(-window_size, window_size + 1)
                if delta != 0 and i + delta >= 0 and i + delta < len(tokens)
            ]
            if len(context) != 2 * window_size:
                continue
            contexts.append((central_word, context))
    return contexts



In [12]:
dataset = pd.read_csv("../lenta-ru-news.csv", sep=',', quotechar='\"', escapechar='\\', encoding='utf-8', header=0)
dataset["date"] = dataset["url"].apply(lambda x: dt.datetime.strptime(get_date(x), "%Y/%m/%d"))
dataset = dataset[dataset["date"] > "2017-01-01"]
dataset["text"] = dataset["text"].apply(lambda x: x.replace("\xa0", " "))
dataset["title"] = dataset["title"].apply(lambda x: x.replace("\xa0", " "))
train_dataset = dataset[dataset["date"] < "2018-04-01"]
test_dataset = dataset[dataset["date"] > "2018-04-01"]

texts = get_texts(train_dataset)
test_texts = get_texts(test_dataset)

assert len(texts) == 827217
assert len(texts[0]) > 0
assert texts[0][0].islower()
print(texts[0])

['возобновление', 'нормального', 'сотрудничества', 'между', 'россией', 'и', 'нато', 'невозможно', 'пока', 'москва', 'не', 'будет', 'соблюдать', 'нормы', 'международного', 'права']


In [13]:
vocabulary = Vocabulary()
vocabulary.build(texts)
assert vocabulary.word2index[vocabulary.index2word[10]] == 10
print(vocabulary.size)
print(vocabulary.top(100))

71186
['в', 'и', 'на', '«', '»', 'что', 'с', 'по', '—', 'не', 'из', 'этом', 'об', 'о', 'он', 'за', 'года', 'россии', 'к', 'его', 'для', 'как', 'также', 'от', 'а', 'это', 'сообщает', 'до', 'году', 'после', 'сша', 'у', 'во', 'время', 'был', 'при', 'заявил', 'со', 'словам', 'рублей', 'будет', 'ее', 'она', 'но', 'ранее', 'их', 'они', 'было', 'тысяч', 'более', 'того', 'том', 'мы', 'были', 'я', 'которые', 'все', 'который', 'человек', 'под', '2016', 'из-за', 'лет', '2017', 'украины', 'марта', 'процентов', 'чтобы', 'долларов', 'глава', 'президент', 'этого', 'отметил', 'же', 'сказал', 'так', 'января', 'или', 'страны', 'ру', 'то', 'еще', 'области', 'данным', 'была', 'президента', 'около', 'сообщил', 'февраля', 'однако', 'компании', 'может', 'уже', 'один', 'рассказал', 'только', 'процента', '1', '10', 'июня']


In [14]:
contexts = build_contexts(texts, vocabulary, window_size=2)
print(contexts[:5])
print(vocabulary.get_word(contexts[0][0]), [vocabulary.get_word(index) for index in contexts[0][1]])

[(1568, [17232, 26343, 135, 371]), (135, [26343, 1568, 371, 2]), (371, [1568, 135, 2, 695]), (2, [135, 371, 695, 2140]), (695, [371, 2, 2140, 216])]
сотрудничества ['возобновление', 'нормального', 'между', 'россией']


In [15]:
with open("../data/prepared.pkl", "wb") as fp:
    pkl.dump(
        file=fp, 
        obj={
            "vocabulary": vocabulary,
            "texts": texts,
            "contexts": contexts,
            "test_texts": test_texts,
        }
    )

In [6]:
with open("../data/prepared.pkl", "rb") as fp:
    prepared = pkl.load(fp)